In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This Notebook had some changes hosted to Ngrok to become an API to call to Frontend

# ColBERTv2: Indexing & Search Notebook

If you're working in Google Colab, we recommend selecting "GPU" as your hardware accelerator in the runtime settings.

First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here. Next, we'll download the necessary dependencies.

In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2417, done.
remote: Counting objects: 100% (937/937), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 2417 (delta 755), reused 699 (delta 691), pack-reused 1480
Receiving objects: 100% (2417/2417), 1.98 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (1504/1504), done.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-cpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/5

In [ ]:
import colbert

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the dev set of the **LoTTE benchmark** we recently introduced in the ColBERTv2 paper. We'll download it from HuggingFace datasets. The dev and test sets contain several domain-specific corpora, and we'll use the smallest dev set corpus, namely lifestyle:dev.

For the purposes of a quick demo, we will only run the `Indexer` on the first 10,000 passages. As we do this, let's also remove the queries whose relevant passages are all outside this small set of passages.

In [ ]:
# from datasets import load_dataset

# dataset = 'lifestyle'
# datasplit = 'dev'

# collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
# collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

# queries_dataset = load_dataset("colbertv2/lotte", dataset)
# queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

# f'Loaded {len(queries)} queries and {len(collection):,} passages'

This loaded 417 queries and 269k passages. Let's inspect one query and one passage to verify we have done so correctly.

In [ ]:
# print(queries[24])
# print()
# print(collection[19929])
# print()

Save this to a file.

In [ ]:
def save_data(data, location):
  # Lưu collection và queries vào file TSV trong thư mục hiện tại
  with open(location, 'w', encoding='utf-8') as file:
    # Ghi header
    file.write("data\n")

    # Ghi dữ liệu
    for item in data:
        file.write(f"{item}\n")
  print("file saved at", location)
  return
def load_data(location):
  loaded_data = []

  with open(location, 'r', encoding='utf-8') as file:
      # Bỏ qua header
      next(file)

      # Đọc dữ liệu
      for line in file:
          item = line.strip()
          loaded_data.append(item)
  return loaded_data



In [ ]:
import os

folder = '/content/drive/MyDrive/CS336'

collection_locate = os.path.join(folder, 'col.tsv')
queries_locate = os.path.join(folder, 'qry.tsv')

#save_data(collection,collection_locate)
#save_data(queries,queries_locate)

collection = load_data(collection_locate)
queries = load_data(queries_locate)

# print(f'Loaded {len(queries)} queries and {len(collection):,} passages')

In [ ]:
print(queries[24])
print()
print(collection[19929])
print()

are blossom end rot tomatoes edible?

I think the spraying thing is not after, its during. The cold will freeze the mist, keeping the air around the trees at (but not below) freezing. See http://www.ehow.com/how_5805520_use-freeze-damage-fruit-trees.html for example which recommends a sprinkler. The releases heat thing is kind of an oversimplification, but basically as long as you have any liquid water around, it will keep things at zero. The sap of your tree is not pure water, and therefore freezes somewhat below zero. By having the water freeze instead you stay away from the temps that would damage your plants. That said, http://www.ehow.com/how-does_5245655_spraying-frost-protect-fruit-freezing_.html is total gibberish since evaporation doesnt generate heat, quite the opposite. There is a better explanation at http://www.gardenguides.com/135830-spray-water-plants-during-frost.html This is a picture from a blog entry that gives you details from the citrus farmers point of view.



## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [ ]:
from datasets import load_dataset

dataset = 'lifestyle'
datasplit = 'dev'

In [ ]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 10000

index_name = f'{dataset}.{datasplit}.{nbits}bits'

To save space and time, we will only run the `Indexer` on the first 10,000 passages. To do so, we will filter out queries that do not contain passages with ids less than 10,000.

In [ ]:
# answer_pids = [x['answers']['answer_pids'] for x in queries_dataset['search_' + datasplit]]
# filtered_queries = [q for q, apids in zip(queries, answer_pids) if any(x < max_id for x in apids)]


# import os

# folder = '/content/drive/MyDrive/CS336'

# filtered_qry_locate = os.path.join(folder, 'qry_filtered.tsv')
# save_data(filtered_queries,filtered_qry_locate)


# f'Filtered down to {len(filtered_queries)} queries'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about six minutes to finish running.

In [ ]:
# checkpoint = 'colbert-ir/colbertv2.0'

# with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
#     config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
#                                                                                 # Consider larger numbers for small datasets.

#     indexer = Indexer(checkpoint=checkpoint, config=config)
#     indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)

In [ ]:
# indexer.get_index() # You can get the absolute path of the index, if needed.
# !cp -r '{indexer.get_index()}' '/content/drive/MyDrive/CS336'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [ ]:
!mkdir -p /content/experiments/notebook/indexes
!cp -r /content/drive/MyDrive/CS336/lifestyle.dev.2bits /content/experiments/notebook/indexes

In [ ]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[Dec 28, 06:29:34] #> Loading codec...
[Dec 28, 06:29:34] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Dec 28, 06:31:08] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Dec 28, 06:32:43] #> Loading IVF...
[Dec 28, 06:32:43] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1058.63it/s]

[Dec 28, 06:32:43] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00,  9.31it/s]


In [ ]:
# query = queries[13] # try with an in-range query or supply your own
# print(f"#> {query}")

# # Find the top-3 passages for this query
# results = searcher.search(query, k=3)
# print(results)
# Print out the top-k retrieved passages
# for passage_id, passage_rank, passage_score in zip(*results):
#     print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

In [ ]:
!pip install ngrok
!pip install pyngrok
!pip install flask
!pip install flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.9 MB/s eta 0:00:00


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-12-28 06:33:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  5.60MB/s    in 2.4s    

2023-12-28 06:33:12 (5.60 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from flask_cors import CORS
import os
import threading

import json
# Đặt authtoken cho Pyngrok
conf.get_default().auth_token = '2HojIOHBoJEI3pFQUsY75937wD1_78srs9y7j1dq54tzpDKrw'

app = Flask(__name__)
CORS(app)

port = "5100"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route('/search', methods=['POST'])
def search():
    try:
        data = request.get_json()
        query = data['query']
        results = searcher.search(query, k=3)
        # Tạo danh sách mục trong định dạng mong muốn-
        result = []  # Khởi tạo danh sách result trước khi sử dụng

        for i in range(3):
            # Đối với mỗi kết quả, tạo một dictionary mới và thêm vào danh sách result
            result.append({
                "rank": results[1][i],  # Lấy rank từ results[1]
                "score": results[2][i],  # Lấy score từ results[2]
                "doc": searcher.collection[results[0][i]]  # Lấy passage_id từ results[0] và sử dụng nó để lấy doc từ collection
            })

        # Chuyển danh sách result thành chuỗi JSON
        result_json = json.dumps(result)
        #json_result = json.dumps(result_list)

        # Trả về chuỗi JSON
        return result_json

    except Exception as e:
        return jsonify({'error': str(e)})


# Start the Flask server in a new thread
threading.Thread(target=app.run(port='5100'), kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://091b-34-87-153-0.ngrok-free.app" -> "http://127.0.0.1:5100"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5100
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 06:58:51] "OPTIONS /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 06:58:51] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:03:24] "OPTIONS /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:03:25] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:04:11] "OPTIONS /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:04:12] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:05:37] "OPTIONS /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:05:38] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:08:16] "OPTIONS /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Dec/2023 07:08:17] "